In [14]:
import pandas as pd


import pandas as pd
import glob

def process_dataset(file):
    d = pd.read_csv(file, sep=",")
    d.columns=['Name', 'Gender', 'Frequency']
    d['Year'] = int(file.split('/')[-1][3:7])
    return d

allfiles = glob.glob('Names/*.txt')
data = pd.concat(process_dataset(file) for file in allfiles)
data.head()


,Name,Gender,Frequency,Year
0,Anna,F,2604,1880
1,Emma,F,2003,1880
2,Elizabeth,F,1939,1880
3,Minnie,F,1746,1880
4,Margaret,F,1578,1880


In [15]:
def top_k_names_in_year(k, year):
    top_k = data.loc[data['Year']==year].nlargest(k, 'Frequency')
    result = {}
    for _, row in top_k.iterrows(): 
        result[row['Name']]= row['Frequency']
    return result

top_k_names_in_year(5, 1905)

{'John': 8060, 'Helen': 6811, 'William': 6495, 'James': 6042, 'Margaret': 5690}

In [16]:
def gender_name_frequencies(name):
    return {
        'M':data.loc[(data['Name']==name) & (data['Gender']=='M')].Frequency.sum(),
        'F':data.loc[(data['Name']==name) & (data['Gender']=='F')].Frequency.sum()
    }

gender_name_frequencies('John')

{'M': 5095674, 'F': 21657}

In [17]:
def top_k_names_in_year_relative_frequency(k, year):
    number_names = len(data.loc[data['Year']==year].Name.unique())
    top_k = data.loc[data['Year']==year].nlargest(k, 'Frequency')
    result = {}
    for _, row in top_k.iterrows(): 
        result[row['Name']]= row['Frequency']/number_names
    return result

top_k_names_in_year_relative_frequency(5, 1905)    

{'John': 2.419693785649955,
 'Helen': 2.044731311918343,
 'William': 1.9498649054338038,
 'James': 1.8138697087961573,
 'Margaret': 1.708195737015911}

In [30]:
def gender_name_frequency_in_year(name, year):
    m_cnt_vals = data.loc[(data['Name']==name) & (data['Gender']=='M') & (data['Year']==year), 'Frequency']
    f_cnt_vals = data.loc[(data['Name']==name) & (data['Gender']=='F') & (data['Year']==year), 'Frequency']
    print(m_cnt_vals, "asdasdsa", f_cnt_vals)
    m_cnt, f_cnt = 0,0
    if m_cnt_vals.size>0:
        m_cnt=m_cnt_vals[:1]
    if f_cnt_vals.size>0:
        f_cnt=f_cnt_vals[:1]  
    return (m_cnt, f_cnt)   

def gender_popularity_shifted_names():
    names = []
    for name in data.Name.unique():
        prev = gender_name_frequency_in_year(name, 1880)
        gender=''
        if prev[0]>prev[1]:
            gender='M'
        else:
            gender='F'
        for year in range(1880, 2016):
            curr = gender_name_frequency_in_year(name, year)
            new_gender ='M' if prev[0]>prev[1] else 'F'
            if new_gender!=gender:
                names.add(name)
                break
            gender = curr
    return names

names_to_count=dict()
names = {}
def foo(row):
    if row.empty():
        return 0
    curr_name = row.iloc[0]['Name']
    if curr_name in names:
        return 0
    elif len(row)==2: 
        male_row = 0 if row.iloc[0]['Gender']=='M' else 1
        female_row = 1 if male_row==0 else 0        
        difference = row.iloc[male_row]['Frequency']-row.iloc[female_row]['Frequency']
        prev_diff = names_to_count[curr_name]
        if prev_diff<0 and difference>0 or prev_diff>0 and difference<0:
            names.add(curr_name)
            return 0
        else:
            names_to_count[curr_name]=0
    return 0
data[:3000].groupby(['Name', 'Year']).apply(func=foo)
print(names)


In [31]:
gender_popularity_shifted_names()

1469    12
Name: Frequency, dtype: int64 asdasdsa 0    2604
Name: Frequency, dtype: int64


ValueError: Can only compare identically-labeled Series objects